In [1]:
!python -V


Python 3.6.8


In [35]:
import spacy
from bs4 import BeautifulSoup
import json
import re
from nltk.tokenize import wordpunct_tokenize, word_tokenize

In [3]:
with open('datasets/foodbase/FoodBase_curated.xml', 'r') as f:
    data = f.read()

Bs_data = BeautifulSoup(data, "xml")

b_doc = Bs_data.find_all('document')


In [4]:
print(type(b_doc[0]))

<class 'bs4.element.Tag'>


In [5]:
b_annotations = b_doc[0].find_all("annotation")
print(b_annotations)

[<annotation id="1">
<location length="12" offset="3"/>
<text>cream cheese</text>
<infon key="semantic_tags"> AG.01.e [Dairy produce];AG.01.e.02 [Cheese];AG.01.n [Dishes and prepared food];AG.01.n.18 [Preserve];</infon>
</annotation>, <annotation id="2">
<location length="4" offset="6"/>
<text>beef</text>
<infon key="semantic_tags"> AG.01.d.03 [Beef];</infon>
</annotation>, <annotation id="3">
<location length="6" offset="8"/>
<text>olives</text>
<infon key="semantic_tags"> AG.01.h.01.e [Fruit containing stone];</infon>
</annotation>, <annotation id="4">
<location length="5" offset="10"/>
<text>onion</text>
<infon key="semantic_tags"> AG.01.h.02.e [Onion/leek/garlic];</infon>
</annotation>, <annotation id="5">
<location length="20" offset="13"/>
<text>Worcestershire sauce</text>
<infon key="semantic_tags"> AG.01.h [Fruit and vegetables];AG.01.l.04 [Sauce/dressing];</infon>
</annotation>, <annotation id="6">
<location length="7" offset="63"/>
<text>walnuts</text>
<infon key="semantic_ta

In [51]:
def get_all_occurences(text, doc):
    return [m.start() for m in re.finditer(text, doc)]

# def get_tokens(sent):
#     tokens = word_tokenize(sent)
#     token_index = []
#     cur_indx = 0
#     for i in range(0, len(tokens)-1):
#         token = tokens[i]
#         token_index.append(cur_indx)
#         cur_indx += len(token) + (0 if tokens[i+1] in [',', '.'] else 1)
    
#     return tokens, token_index

def get_tokens(txt):
    tokens = word_tokenize(txt)
    offset = 0
    token_indx_tuples = []
    for token in tokens:
        offset = txt.find(token, offset)
        token_indx_tuples.append((token, offset))
        offset += len(token)
    
    return token_indx_tuples


In [ ]:
data = []
i = 0
for docs in b_doc:
    infon = docs.find("infon", {"key": "full_text"})
    document = infon.get_text().strip()
    data.append([])
    data[i].append(document)
    data[i].append({'entities': []})
    current_end = 0
    token_indx_tuples = get_tokens(document)
    for anots in docs.find_all("annotation"):
        word = anots.find("text").get_text()
        place = anots.find("location")
        offset = place["offset"] # find the first occurence of this
        token, starting_index = token_indx_tuples[int(offset) - 1]
        length = place["length"]
        end = int(starting_index) + int(length)
        print(document[starting_index:end])
        data[i][1]["entities"].append([starting_index, end, "INGREDIENT"])
        current_end = end
    i+=1
    # break
    

cream cheese
beef
olives
onion
Worcestershire sauce
walnuts
cheese ball
walnuts
cheese ball
artichoke hearts
mozzarella cheese
parmesan cheese
mayonnaise
almonds
tortilla chips
crackers
water
sun-dried tomatoes
water
cream cheese
garlic
provolone cheese
pesto
cream cheese
garlic mixture
sun-dried tomatoes
provolone
pesto
cream cheese
garlic mixture
sun-dried tomatoes
provolone cheese
excess oil
garlic
salt
garbanzo beans
tahini
lemon juice
honey
water
beans
olive oil
pita breads
olive oil
salt
fresh rosemary
hummus
shredded chicken
picante sauce
ground cumin
vegetable oil
corn tortilla
oil
tortillas
tortillas
oil
chicken mixture
tortilla
cheese
tortilla
tortillas
chicken mixture
cheese
rolled tortillas
tortillas
tortillas
cream cheese
meat slice
pickle piece
meat
dry ranch salad dressing mix
mayonnaise
milk
cream cheese
Cheddar cheese
almonds
rice
cold water
ground lamb
rice
allspice
salt
pepper
meat mixture
leaf
leaf
leaf
leaf-rolls
garlic
water
rolls
lemon juice
olives
rolls
rice
Gra

pecans
coated pecans
extra egg white mixture
pepper mixture
pecan
spices
pecans
pecans
pecans
pecans
ground beef
onion
seasoned bread crumbs
water
soy sauce
garlic
whiskey
sauce mixture
cooking spray
butter
white corn syrup
brown sugar
butter
cereal
pecans
almonds
melted butter mixture
cereal
nuts
cereal
nuts
snack mix
breadcrumbs
crab
shrimp
green onions
celery
mayonnaise
Worcestershire sauce
parsley
seafood shells
stuffed shells
bread crumbs
salted water
pasta
boiling water
crab
noodles
eggs
green onion
bread crumbs
cilantro
olive oil
salt
pepper
patty
noodles
vegetable oil
patties
cilantro-lime mayonnaise
cilantro-lime mayonnaise
mayonnaise
cilantro
lime zest
lime juice
refried beans
taco seasoning mix
tomato
Cheddar cheese
eggs
water
water
water
eggs
hot water
hot water
eggs
water
eggs
yolks
yolks
sugar
salt
onion powder
garlic powder
white pepper
mustard
mayonnaise
salad dressing
vinegar
yolk mixture
egg half
paprika
cooking spray
cauliflower
olive oil
garlic powder
cauliflower mi

cream cheese
sour cream
cumin
garlic salt
hot pepper sauce
tomato
onion
lettuce
Cheddar cheese
water
egg
pepper
pancake mix
milk
corn
oil
fritter
oil
eggplant
onion
garlic
oregano
cinnamon
salt
pepper
onion
tomatoes
water
wine
bay leaf
eggplant
sugar
capers
vinegar
bay leaf
ground beef chuck
Cheddar cheese
mayonnaise
dry onion soup mix
ground beef
roll
sandwiches
rolls
meat
corn kernels
tomatoes
hot yellow banana pepper
garlic
green bell pepper
basil
sage
thyme
lemon juice
olive oil
olive oil
parsley
lemon juice
hot sauce
garlic
tomato paste
oregano
salt
black pepper
marinade
shrimp
shrimp
marinade
shrimp
reserved marinade
refried beans
hot bean dip
bean mixture
avocados
salsa
bean mixture
sour cream
shredded Cheddar cheese
black olives
diced tomatoes
green onions
dip
mango
papaya
onion
garlic
jalapeno pepper
red bell pepper
orange bell pepper
lime juice
salt
pepper
potato nuggets
potato nuggets
potato nuggets
pepperjack cheese
pulled pork
onion
jalapeno peppers
barbeque sauce
totchos


eggs
whipping cream
salt
flour
cheese sandwiches
cheese slice
bread slices
sandwich
beaten eggs
milk
salt
pepper
sandwiches
grated cheese
sandwiches
egg mixture
egg
pie crust
crust
crust
butter
zucchini
carrots
mushrooms
vegetables
zucchini mixture
crust
mozzarella cheese
ricotta cheese
eggs
salt
oregano
basil
garlic powder
pepper
ricotta mixture
crust
zucchini mixture
paprika
quiche
quinoa flour
baking powder
salt
flaxseed milk
eggs
melted butter
honey
flour mixture
thin batter
batter
pancake
pancake
flour
strawberries
sugar
whole wheat flour
all-purpose flour
oats
white sugar
baking powder
cinnamon
bananas
eggs
oil
strawberry mixture
flour mixture
batter
batter
butter
foam
onion
tomato
onion
liquid
tomato
tomato
onion
beaten eggs
cooked vegetables
tarragon
basil
salt
black pepper
Cheddar cheese
egg mixture
bell pepper
pepper
pepper
eggs
unpeeled potatoes
cold water
potatoes
water
potatoes
potatoes
potatoes
butter
sugar
mashed potatoes
flour
potatoes
flour
mashed potatoes
flour
lefse 

cubed Cheddar cheese
onion
eggs
evaporated milk
salt
pepper
garlic powder
parsley
sage
salmon
pie crust
shredded Cheddar cheese
egg mixture
shredded Cheddar cheese
pie crust
broccoli
broccoli
butter
onion
garlic
melted butter
ham
broccoli
onion mixture
pie crust
Cheddar cheese
milk
eggs
black pepper
ham mixture
paprika
entire quiche
quiche
pie pasty
pie crust
crust
Cheddar cheese
Swiss cheese
flour
warm pie crust
ham
honey mustard
ham
cheese mixture
eggs
green onions
flour
salt
Cheddar cheese
milk
egg yolks
vanilla
flour
sugar
salt
melted butter
vegetable oil
cooking spray
batter
batter
batter
crepes
fruit
cream
caramel
ice cream
cheese
baking soda
boiling water
dates
water
baking soda solution
sugar
butter
eggs
vanilla
dates
nuts
flour
salt
loaf
butter
green onion
red bell pepper
crabmeat
salt
pepper
wine
eggs
crab mixture
pie crust
Swiss cheese
milk
instant breakfast mix
peanut butter
bananas
ice
pancake mix
apple cider
Apple cider
cinnamon
nutmeg
brown sugar
vanilla
walnuts
cooking 

butter pieces
dough
apple mixture
dough
crusts
dough
dough
pie
pie
apples
filling
crust
chocolate cake mix
water
vegetable oil
eggs
rhubarb
cherry gelatin mix
sugar
rhubarb
cake batter
strawberries
brown sugar
flour
white sugar
salt
butter
flour mixture
coarse crumbs
milk
dough
dough
dough
dough
dry beans
crust
crust
cream
egg yolks
salt
hot cream
beaten yolks
hot cream
yolks
cream
yolk mixture
hot cream
cream mixture
custard
custard
juice
strawberries
baked pie shell
cream custard
berries
white sugar
pie
sugar
pie
sugar topping
boiling water
tea bags
tea bags
sugar
lime juice
lime slice
ice pop mold
warm water
ice pops
white sugar
butter
eggs
vanilla
flour
baking powder
flour mixture
milk
batter
cake
brown sugar
egg whites
water
salt
boiling water
frosting
frosting
vanilla
frosting
cooled cake
large raisin
toasted almonds
toasted pecans
green cherries
cake
graham crackers
crumbs
tortilla
melted butter
cracker crumbs
tortillas
chips
chips
chocolate sauce
marshmallows
marshmallows
apple

walnuts
pecans
hazelnuts
raisins
coconut
sugar
cooked pastry
nut mixture
milk
sugar
nut mixture
heavy cream
sugar
nut mixture
dessert
cream cheese
sugar
eggs
extracts
sour cream
batter
cake
canned pie filling
unsweetened chocolate
water
butter
melted chocolate
sugar
egg
vanilla
flour
baking soda
chocolate chips
macadamia nuts
dough
margarine
eggs
confectioners' sugar;
graham cracker crusts
crust
banana slices
sugar mixture
pineapple
bananas
reserved juice
fruit
whipped topping
fruit
cherries
pecans
corn flakes
coconut
peanut butter
honey
cornflakes
cookies
butter
margarine
eggs
sugar
sour cream
vanilla
melted butter
flour
baking soda
nutmeg
egg mixture
chocolate
butterscotch chips
raisins
peanuts
marshmallows
chow mein noodles
melted chocolate mixture
peanut butter
butter
marshmallows
honey
honey graham cereal
shortening
brown sugar
egg
milk
vanilla
oats
flour
baking soda
salt
cinnamon
creamed mixture
nuts
caramel
caramel
butter
margarine
flour
sugar
milk
peaches
syrup
crust
unflavored

ice cubes
peppermint candies
chocolate chips
ice cream
peppermint extract
red food coloring;
milk
ice cubes
vodka
cranberry juice
grape juice
soda
butterscotch schnapps
Irish liqueur
cinnamon schnapps
crushed ice
sugar
lime gelatin mix
boiling water
gelatin mixture
sugar
gelatin
chill gelatin mixture
lime gelatin mixture
pineapple juice
lemon juice
almond extract
cold water
gin
lemon juice
honey syrup
ice
protein powder
flax meal
cocoa powder
brown sugar
almond milk
banana
berries
peanut butter
orange juice
daiquiri mixer
lemon-lime soda
vodka
hot chocolate Irish cream liqueur
creme de cacao
tea bags
mint sprigs
boiling water
tea bag mixture
tea
steeped tea
excess liquid
tea bag
tea bag
sugar
tea
lemon juice
lime juice
vanilla extract
cold water
tea mixture
ice
tea
ice
raspberry-flavored vodka
lemon-flavored vodka
simple syrup
lemon juice
raspberry-flavored liqueur
drink
kale
soy milk
spinach
banana
raspberries
pineapple
graham cracker crumbs
graham cracker crumbs
ice
chocolate liqueur

In [46]:
with open("FoodBase_curated_fixed.json", 'w') as outfile:
    json.dump(data, outfile)

In [47]:
with open("FoodBase_curated_fixed.json") as json_file:
    data = json.load(json_file)
print(data[0])

['Mix the cream cheese, beef, olives, onion, and Worcestershire sauce together in a bowl until evenly blended. Keeping the mixture in the bowl, scrape it into a semi-ball shape. Cover, and refrigerate until firm, at least 2 hours. Place a large sheet of waxed paper on a flat surface. Sprinkle with walnuts. Roll the cheese ball in the walnuts until completely covered. Transfer the cheese ball to a serving plate, or rewrap with waxed paper and refrigerate until needed.', {'entities': [[315, 326, 'INGREDIENT'], [381, 392, 'INGREDIENT'], [28, 34, 'INGREDIENT'], [8, 20, 'INGREDIENT'], [47, 67, 'INGREDIENT'], [297, 304, 'INGREDIENT'], [334, 341, 'INGREDIENT'], [36, 41, 'INGREDIENT'], [22, 26, 'INGREDIENT']]}]
